In [58]:
# Get the reuters data
from nltk.corpus import reuters

# overall algo:
# create a free flowing text without any punctuation etc. only alphabet
# reuters data consists of sentences; each sentence is a list of words
# so first of all combine all the words into one text string; then apply cleaning (only alphabet, 
# lower case, ' u  s ' to 'usa')

# ok step 1
# combine all words of all sentences 
sentLimit = 1000
sentCnt = 0
allText = ''
for sentence in reuters.sents():
    sentCnt += 1
    if sentCnt <= sentLimit:
        for word in sentence:
            allText += word + ' '
    else:
        break
# replace " ' s '" by " "
import re
allText = re.sub(" ' s ", " ", allText)
allText = re.sub(r'([^a-zA-Z])', " ", allText)
allText = allText.lower()
allText = re.sub(" u   s ", " usa ", allText)
allText = re.sub("  ", " ", allText)
allText = " ".join(allText.split())

# now create sequences of seqLen (=5 say) characters where first four will be used for input and 5th char for output
allSeq = []
seqLen = 30
for i in range(0, len(allText)-seqLen+1):
    allSeq.append(allText[i:i+seqLen])

# create a mapping of each character to an integer
map1 = sorted(list(set(allText)))
mapping = dict((c, i) for i, c in enumerate(map1))

# now convert allSeq using this mapping
allIntSeq = []
for i in range(len(allSeq)):
    allIntSeq.append([mapping[ch1] for ch1 in allSeq[i]])

# using numpy create X and y from allIntSeq
import numpy as np
X, y = np.array(allIntSeq)[:, :-1], np.array(allIntSeq)[:, -1]

# one-hot encode y to vector of lengthlen(map1)
from keras.utils import to_categorical
y = to_categorical(y, len(map1))

# split X and y into train and test
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# now build the LSTM model: first import the libraries
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

model = Sequential()
model.add(Embedding(len(map1), 50, input_length=29, trainable=True))
model.add(LSTM(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(len(map1), 'softmax'))
#print(model.summary)
model.compile(loss='categorical_crossentropy', metrics='acc', optimizer='adam')
model.fit(X_tr, y_tr, epochs=10, verbose=1, validation_data=(X_val, y_val))



Epoch 1/10
372/372 [==============================] - 68s 154ms/step - loss: 2.7565 - acc: 0.2048 - val_loss: 2.4701 - val_acc: 0.2771
Epoch 2/10
372/372 [==============================] - 60s 161ms/step - loss: 2.1348 - acc: 0.3562 - val_loss: 2.1137 - val_acc: 0.3520
Epoch 5/10
372/372 [==============================] - 57s 154ms/step - loss: 2.0287 - acc: 0.3850 - val_loss: 2.0429 - val_acc: 0.3687
Epoch 6/10
372/372 [==============================] - 55s 148ms/step - loss: 1.9312 - acc: 0.4181 - val_loss: 1.9808 - val_acc: 0.4103
Epoch 7/10
372/372 [==============================] - 52s 139ms/step - loss: 1.7554 - acc: 0.4724 - val_loss: 1.8853 - val_acc: 0.4368
Epoch 9/10
372/372 [==============================] - 57s 153ms/step - loss: 1.6797 - acc: 0.5014 - val_loss: 1.8497 - val_acc: 0.4504
Epoch 10/10
372/372 [==============================] - 54s 145ms/step - loss: 1.6035 - acc: 0.5201 - val_loss: 1.8139 - val_acc: 0.4693


In [45]:
allText

'asian exporters fear damage from usa japan rift mounting trade friction between the usa and japan has raised fears among many of asia exporting nations that the row could inflict far reaching economic damage businessmen and officials said'

In [63]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
# make prediction using seed_text
seed_text="but som"
numPred = 10

# make numPred predictions
for i in range(numPred):
    # convert to integers using mapping
    seedSeq = [mapping[ch1] for ch1 in seed_text]
    paddedSeq = pad_sequences([seedSeq], seqLen-1, truncating='pre')
    yhat_probs = model.predict(paddedSeq)
    for ch1, ind in mapping.items():
        if ind == np.argmax(yhat_probs):
            yhat = ch1
    seed_text += yhat
print(seed_text)

1/1 [==============================] - 0s 56ms/step
but some the said
